In [1]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from glob import glob
from time import sleep

**1). Необходимо собрать информацию о вакансиях на должность программиста или разработчика с сайта superjob.ru или hh.ru. (Можно с обоих сразу) Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе: **

 - Наименование вакансии,
 - Предлагаемую зарплату,
 - Ссылку на саму вакансию

**2). Доработать приложение таким образом, чтобы можно было искать разработчиков на разные языки программирования (Например Python, Java, C++)**

In [2]:
columns = ['vacancy', 'snippet', 'salary', 'employer', 'vacancy_link']
hh = pd.DataFrame(columns=columns)
sj = pd.DataFrame(columns=columns)

Через парсинг сохраненных страниц сайта hh.ru получим данные по вакансиям:

In [3]:
for file_name in glob(os.path.join('hh', '*.html')):
    with open(file_name, 'rb') as f:
        parser = bs(f, 'html.parser')
        vacancies = parser.find_all('div', {'class': 'vacancy-serp-item'})
        for v in vacancies:
            vacancy = v.find('a', {'class': 'bloko-link HH-LinkModifier'}, href=True)
            snippet = v.find('div', {'data-qa': "vacancy-serp__vacancy_snippet_requirement"})
            salary_info = v.find('div', {'class': "vacancy-serp-item__compensation"})
            if salary_info:
                salary = salary_info.get_text()
            else:
                salary = '-'
            employer = v.find('a', {'data-qa': "vacancy-serp__vacancy-employer"})
            vacancy_link = vacancy['href']
            vacancy_info = {
                'vacancy': vacancy.get_text(),
                'snippet': snippet.get_text(),
                'salary': salary,
                'employer': employer.get_text(),
                'vacancy_link': vacancy_link,
            } 
            hh = hh.append(vacancy_info, ignore_index=True)

Поправим немного ссылки:

In [4]:
hh['vacancy_link'] = hh['vacancy_link'].str.replace('[?]query=.+', '', regex=True)

In [5]:
hh.head()

,vacancy,snippet,salary,employer,vacancy_link
0,Программист С++,Отличное знание C++. Умение разбираться в чужо...,60 000-100 000 руб.,Cappasity Inc.,https://hh.ru/vacancy/32323917
1,Программист С++/Qt,"QT framework. Знание паттернов, разработка мно...",60 000-100 000 руб.,Cappasity Inc.,https://hh.ru/vacancy/31273030
2,Full-Stack Разработчик JavaScript (Node.js + A...,JavaScript / TypeScript (идеальное владение). ...,от 120 000 руб.,ИП Бушев Юрий Владимирович,https://hh.ru/vacancy/32859767
3,Программист (C#\ ASP.NET\MVC),Высшее профильное образование. Опыт разработки...,от 100 000 руб.,ООО МИЮИ,https://hh.ru/vacancy/33036570
4,Web-программист/PHP-программист,"Знание PHP, MySQL‚ HTML‚ CSS‚ JavaScript. - Оп...",-,Онлишар,https://hh.ru/vacancy/32728996


Через запросы на сайт superjob.ru и парсинг результатов запроса получим данные по вакансиям:

In [6]:
for page in ['', '?page=2', '?page=5']:
    url = 'https://www.superjob.ru/vakansii/programmist.html' + page
    sleep(6)
    r = requests.get(url)
    print(r)
    parser = bs(r.text, 'html.parser')
    vacancies = parser.find_all('div', {'class': '_3zucV _2GPIV i6-sc _3VcZr'})
    for v in vacancies:
        vacancy = v.find('div', {'class': '_3mfro CuJz5 PlM3e _2JVkc _3LJqf'})
        salary = v.find('span', {'class': 'f-test-text-company-item-salary'}).get_text()
        salary = salary.replace('По договорённости', '')
        employer_info = v.find('span', {'class': 'f-test-text-vacancy-item-company-name'})
        if employer_info:
            employer = employer_info.get_text()
        else:
            employer = '-'
        snippet_info = v.find_all('span', {'class': '_3mfro _9fXTd _2JVkc _2VHxz'})
        if len(snippet_info) == 2:
            snippet = snippet_info[0].get_text().replace('Должностные обязанности: ', '')
            snippet += snippet_info[1].get_text().replace('Требования:', '')
        else:
            snippet = '-'
        vacancy_link = v.find('a', {'class': '_1QIBo'}, href=True)['href']
        vacancy_link = 'https://www.superjob.ru' + vacancy_link
        vacancy_info = {
                    'vacancy': vacancy.get_text(),
                    'snippet': snippet,
                    'salary': salary,
                    'employer': employer,
                    'vacancy_link': vacancy_link,
                }
        sj = sj.append(vacancy_info, ignore_index=True)

<Response [200]>
<Response [200]>
<Response [200]>


Поправим немного зарплату:

In [7]:
sj['salary'] = sj['salary'].str.replace('По договорённости', '-', regex=False)
sj['salary'] = sj['salary'].str.replace('₽', 'руб.', regex=False)

In [8]:
sj.head()

,vacancy,snippet,salary,employer,vacancy_link
0,Web-программист,Центр по разработке ПО в сфере логистики пригл...,150 000 руб.,Кадровое агентство уникальных специалистов,https://www.superjob.ru/vakansii/web-programmi...
1,Программист 1С,Разработка и внедрение программного продукта в...,от 100 000 руб.,Группа компаний ЦВТ,https://www.superjob.ru/vakansii/programmist-1...
2,Программист 1С,Центр по разработке ПО в сфере логистики пригл...,150 000 руб.,Кадровое агентство уникальных специалистов,https://www.superjob.ru/vakansii/programmist-1...
3,Программист 1С (Производство),Поддержка нетиповой конфигурации на базе Управ...,120 000 руб.,Группа компаний ПиР,https://www.superjob.ru/vakansii/programmist-1...
4,Программист-разработчик 1С,Участие в разработке и внедрениях сложных (в т...,от 120 000 руб.,Device PRO,https://www.superjob.ru/vakansii/programmist-r...


Объединим в датафрейм полученные данные с hh.ru и superjob.ru:

In [9]:
df = pd.concat([hh, sj], ignore_index=True)

Напишем функцию, которая фильтрует записи датафрейма по названию языка в столбцах с названием и описанием вакансии.
Сортируем их по зарплате - по убыванию:

In [10]:
def filter_vacancies(l, df):
    idx = df['vacancy'].str.contains(l, regex=False) | \
          df['snippet'].str.contains(l, regex=False)
    return df[idx].sort_values(by='salary', ascending=False)

Отфильтруем в датафрейме вакансии по С++:

In [11]:
filter_vacancies('С++', df=df)

,vacancy,snippet,salary,employer,vacancy_link
72,Программист C++,Поддержка и дальнейшее развитие проектов для а...,от 120 000 руб.,СТОЛИЧКИ,https://www.superjob.ru/vakansii/programmist-c...
136,Ведущий инженер-программист C/C++,Разработка программного обеспечения управления...,85 000 — 95 000 руб.,НРТБ,https://www.superjob.ru/vakansii/veduschij-inz...
5,Программист C/С++,"Обязательно: Уверенное владение C++, STL. Хоро...",80 000-120 000 руб.,АО РАКУРС,https://hh.ru/vacancy/32666256
8,Программист iOS (Swift),Хорошее знание Swift. Желательно знание С++. Ж...,80 000-120 000 руб.,Cappasity Inc.,https://hh.ru/vacancy/31399856
37,Программист С++ (графика),Отличное знание C++. Желательно знакомство с Q...,80 000-120 000 руб.,Cappasity Inc.,https://hh.ru/vacancy/32543357
135,Инженер-программист,Разработка отдельных модулей специального прог...,75 000 — 85 000 руб.,НРТБ,https://www.superjob.ru/vakansii/inzhener-prog...
47,Программист Android,"Отличное знание Java, Kotlin. Желательно знани...",70 000-120 000 руб.,Cappasity Inc.,https://hh.ru/vacancy/32379471
1,Программист С++/Qt,"QT framework. Знание паттернов, разработка мно...",60 000-100 000 руб.,Cappasity Inc.,https://hh.ru/vacancy/31273030
0,Программист С++,Отличное знание C++. Умение разбираться в чужо...,60 000-100 000 руб.,Cappasity Inc.,https://hh.ru/vacancy/32323917
119,Руководитель группы программистов,Распределение работы между подчинёнными и конт...,200 000 руб.,Торговая Сеть Управдом,https://www.superjob.ru/vakansii/rukovoditel-g...
